In [1]:
import copy
import pandas as pd
from functools import reduce
import numpy as np

## Месяц от 1 до 12

In [32]:
setted_month = 6
l = 25

### Загружаем данные

In [33]:
data = pd.read_csv("weather_2017.csv")
data[data['month'] == setted_month]

,Unnamed: 0,month,day,temp_dailyMin,temp_minGround,temp_dailyMean,temp_dailyMax,temp_7h,temp_14h,temp_19h,...,hum_14h,hum_19h,sun_hours,skyCover_7h,skyCover_14h,skyCover_19h,wind_degrees,wind_mSec,precip,year
151,151,6,1,19.0,15.7,23.0,27.0,20.1,26.0,23.4,...,42.0,46.0,10.9,7,6,8,318.0,9.3,0.0,2017
152,152,6,2,16.3,11.2,22.2,28.0,20.2,27.1,24.4,...,40.0,55.0,13.0,3,3,3,93.0,7.7,0.0,2017
153,153,6,3,16.6,12.5,22.7,28.7,21.2,27.4,NaN,...,40.0,59.0,12.6,4,5,4,205.0,11.1,0.0,2017
154,154,6,4,16.9,12.3,22.0,27.0,22.4,23.3,17.1,...,53.0,88.0,6.5,7,10,9,269.0,18.4,2.4,2017
155,155,6,5,12.6,7.9,19.1,25.5,17.9,24.1,22.0,...,42.0,56.0,13.6,3,1,3,120.0,8.2,0.0,2017
156,156,6,6,15.5,11.4,21.6,27.7,NaN,27.4,20.5,...,46.0,69.0,8.0,4,9,10,273.0,21.2,0.7,2017
157,157,6,7,14.5,12.0,17.5,20.5,14.9,18.7,16.3,...,43.0,57.0,1.6,9,9,7,297.0,11.1,0.7,2017
158,158,6,8,11.7,6.8,18.0,24.2,16.2,22.2,20.3,...,37.0,39.0,14.8,1,6,4,287.0,9.9,0.0,2017
159,159,6,9,11.5,5.7,19.5,27.4,17.6,26.6,24.0,...,34.0,40.0,14.6,3,2,8,126.0,12.4,4.8,2017
160,160,6,10,15.5,13.0,20.5,25.4,63.2,23.9,21.7,...,39.0,41.0,8.7,9,5,6,320.0,15.0,4.8,2017


### Создание таблички с данными, по которым будем стоить прогноз
Для начала будем прогнозировать температуру для второй половины января по методу скользящего среднего

In [36]:
data_day = pd.DataFrame()
data_day['year']=data['year']
data_day['month']=data['month']
data_day['day']=data['day']
data_day['temp_dailyMean'] = data['temp_dailyMean']

data_predict = copy.copy(data_day[data_day['month']==setted_month])

### Считаем количество дней в каждом месяце

In [35]:
count_last_day=[]
for i in range(1,13):
    count_last_day.append(data_day[data_day['month']==i]['day'].values[-1])
count_last_day

[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

### Обрезанем данные за январь, чтобы получить данные на которых будем сторить предсказания

In [37]:
#data_sres = data_day.iloc[:int(count_last_day[setted_month-1]//2)]
data_day[data_day['month']==setted_month]

,year,month,day,temp_dailyMean
151,2017,6,1,23.0
152,2017,6,2,22.2
153,2017,6,3,22.7
154,2017,6,4,22.0
155,2017,6,5,19.1
156,2017,6,6,21.6
157,2017,6,7,17.5
158,2017,6,8,18.0
159,2017,6,9,19.5
160,2017,6,10,20.5


### Рассчет скользящего среднего для второй половины месяца

In [62]:
for j in range (count_last_day[setted_month-1]//2, count_last_day[setted_month-1]):
    i = 0
    avg = 0
    weight = 1
    avg_list = []
    while i + l <= count_last_day[setted_month-1]:
        avg_list.append(data_predict.iloc[i:(i+l)]['temp_dailyMean'].mean() ) #* weight)
        #weight *= 0.1
        i+=1
        
    #avg = reduce(lambda x, y: x + y, avg_list)/len(avg_list)
    avg = np.mean(avg_list) 
    
    data_predict.iloc[j] = [2017, setted_month, int(j+1), avg]

#legacy solution
#for i in range(count_last_day[setted_month-1]//2, count_last_day[setted_month-1]):
#    data_predict.iloc[i] = [2017, setted_month, int(i+1), data_predict.iloc[:(i-1)]['temp_dailyMean'].mean()]

data_predict

,year,month,day,temp_dailyMean
151,2017,6,1,23.000000
152,2017,6,2,22.200000
153,2017,6,3,22.700000
154,2017,6,4,22.000000
155,2017,6,5,19.100000
156,2017,6,6,21.600000
157,2017,6,7,17.500000
158,2017,6,8,18.000000
159,2017,6,9,19.500000
160,2017,6,10,20.500000
